In [1]:
import numpy as np
import os
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import confusion_matrix
import tensorflow.keras as keras
from tensorflow.keras import Sequential
from tensorflow.keras import regularizers
from tensorflow.keras.layers import LSTM, Dense, LSTM, Flatten, BatchNormalization, Dropout
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
import module
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [2]:
countriesOfInterest = {"HK", "JP", 'ZA', 'TN', 'TR', 'GB', 'MX', 'US', 'CO', 'EC', 'AU', 'NZ'}
countriesOfInterest = ["ZA", "EG", "TW", "JP", "DK", "FI", "US", "CA", "AU", "NZ", "BR", "CO"]
#countriesOfInterest = list(countriesOfInterest & {"ZA", "EG", "TW", "JP", "DK", "FI", "US", "CA", "AU", "NZ", "BR", "CO"})

enc = OneHotEncoder()
enc.fit(np.array(countriesOfInterest).reshape(-1, 1))

OneHotEncoder(categories='auto', drop=None, dtype=<class 'numpy.float64'>,
              handle_unknown='error', sparse=True)

In [3]:
seconds = 10
samplerate = 100
train_x, train_labels, val_x, val_labels, class_weights = module.getSamples(1, 1, seconds, samplerate, countriesOfInterest,
               enc, verbose = 0)

### Fit model

In [4]:
model = keras.Sequential()
model.add(LSTM(2, 
               input_shape=(train_x.shape[1], train_x.shape[2]), 
               return_sequences = True,
               recurrent_dropout = .5,
              # kernel_regularizer=regularizers.l2(0.01),
              ))
model.add(BatchNormalization())
model.add(LSTM(len(enc.categories_[0]), 
               activation= "softmax",
               #input_shape=(train_x.shape[1], train_x.shape[2]), 
               return_sequences = False,
               recurrent_dropout = .5,
              # kernel_regularizer=regularizers.l2(0.01),
              ))
adam = keras.optimizers.Adam(lr=0.001)
model.compile(loss = "categorical_crossentropy", optimizer= adam, metrics=["acc"])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 1000, 2)           224       
_________________________________________________________________
batch_normalization (BatchNo (None, 1000, 2)           8         
_________________________________________________________________
lstm_1 (LSTM)                (None, 12)                720       
Total params: 952
Trainable params: 948
Non-trainable params: 4
_________________________________________________________________
None


In [ ]:
#add recurrent dropout to add noise to duration
desc = "2LSTM_2_Output_NoReg_3Step_50train_01LR_ChangeStack"
log_dir = os.path.join(
    "logs",
    "betterSampling",
    desc
)
model_dir = os.path.join(
    "pickle",
    "save"
)

train_n = 100
val_n  = int(train_n/5)
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
epochs = 3
iterations = 10
learn_rate = 0.01
module.train(iterations, learn_rate, train_n, val_n, seconds, samplerate,
                countriesOfInterest, enc, epochs, tensorboard_callback, model_dir,
                model)

[4644. 5093. 4208. 5442. 3931. 4528. 4323. 5677. 4847. 4649. 3791. 5372.]
Train on 56505 samples, validate on 6373 samples
Epoch 1/3
10240/56505 [====>.........................] - ETA: 1:50 - loss: 2.4876 - acc: 0.0830